<a href="https://colab.research.google.com/github/Arinjay11000/Minor-Projects/blob/main/Fake%20News%20Classifier/Fake_News_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.set_index("id",inplace=True)
df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18285 non-null  object
 1   author  18285 non-null  object
 2   text    18285 non-null  object
 3   label   18285 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 714.3+ KB


In [ ]:
df.index=[i for i in range(18285)]
df.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df["Merged"]=df["title"]+" "+df["author"]
df.head()

,title,author,text,label,Merged
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [ ]:
df.drop(["title","text","author"],axis=1,inplace=True)
df.head()

,label,Merged
0,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,1,Why the Truth Might Get You Fired Consortiumne...
3,1,15 Civilians Killed In Single US Airstrike Hav...
4,1,Iranian woman jailed for fictional unpublished...


In [ ]:
w=WordNetLemmatizer()
for i in range(len(df)):
  review=re.sub('[^a-zA-Z]', ' ', df["Merged"][i])
  review=review.lower()
  review=review.split()
  review=[w.lemmatize(word) for word in review if not word in set(stopwords.words("english"))]
  review=" ".join(review)
  df["Merged"][i]=review
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,label,Merged
0,1,house dem aide even see comey letter jason cha...
1,0,flynn hillary clinton big woman campus breitba...
2,1,truth might get fired consortiumnews com
3,1,civilian killed single u airstrike identified ...
4,1,iranian woman jailed fictional unpublished sto...


In [ ]:
voc_size=20000
onehot_repr=[one_hot(words,voc_size)for words in df["Merged"]] 
print(onehot_repr)

[[4696, 19424, 4801, 16724, 7477, 4122, 7929, 16426, 16194, 4117, 2004, 6153], [13689, 3706, 13053, 10380, 19838, 10477, 19359, 14636, 14978, 13689], [14589, 17585, 14887, 4425, 7871, 16368], [9817, 9834, 7904, 13332, 5923, 19730, 13421, 12241], [7846, 19838, 8838, 12162, 542, 6540, 19838, 14091, 13647, 14378, 19907, 6225], [14163, 19030, 3680, 16890, 17717, 1923, 13166, 14177, 6174, 19019, 11593, 18742, 3697, 4626, 19359, 14636, 12852], [11079, 7479, 1717, 11091, 13540, 9194, 1466, 4266, 10565, 16071, 1367, 2061, 14978, 1376], [7525, 15423, 15003, 16443, 5322, 16096, 1923, 14150, 10565, 16071, 1367, 13014, 12128, 19073, 10487], [18406, 3313, 14808, 1092, 10924, 8601, 18888, 1315, 1923, 9256, 12880, 10385], [18068, 14177, 9350, 2104, 5863, 4018, 1233, 12481, 15876, 1775], [8810, 9378, 18206, 17494, 3035, 11759, 4886, 76, 11769, 12482, 10418, 10258, 18991], [13332, 15260, 7477, 15531, 1923, 5322, 16426, 1421], [2296, 11085, 3312, 14810, 6995, 18456, 11692, 13726, 6921, 17154], [1235, 15

In [ ]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ...  4117  2004  6153]
 [    0     0     0 ... 14636 14978 13689]
 [    0     0     0 ...  4425  7871 16368]
 ...
 [    0     0     0 ...  5185 14120 12537]
 [    0     0     0 ...  9678 15430 19332]
 [    0     0     0 ...  7126  1711 19149]]


In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 40)            800000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 856,501
Trainable params: 856,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y=df["label"]

In [ ]:
embedded_docs.shape

(18285, 30)

In [ ]:
Y.shape

(18285,)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(embedded_docs, Y, test_size=0.33, random_state=10,stratify=Y)

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 59ms/step - loss: 0.2189 - accuracy: 0.9038 - val_loss: 0.0392 - val_accuracy: 0.9881
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0156 - accuracy: 0.9959 - val_loss: 0.0228 - val_accuracy: 0.9937
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0232 - val_accuracy: 0.9930
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 0.0282 - val_accuracy: 0.9924
Epoch 5/10
192/192 [==============================] - 10s 54ms/step - loss: 4.2917e-04 - accuracy: 1.0000 - val_loss: 0.0281 - val_accuracy: 0.9922
Epoch 6/10
192/192 [==============================] - 10s 54ms/step - loss: 1.0579e-04 - accuracy: 1.0000 - val_loss: 0.0320 - val_accuracy: 0.9920
Epoch 7/10
192/192 [==============================] - 10s 53ms/step - loss: 6.0316e-05 - accuracy: 1.0000 - val_loss: 0.0375 - v

In [ ]:
Y_pred=model.predict(X_test)

In [ ]:
Y_pred=Y_pred>0.5
confusion_matrix(Y_test,Y_pred)

array([[3398,   22],
       [  36, 2579]])

In [ ]:
Y_pred

array([[ True],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [ True]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9903893951946976